## Clone Project

In [ ]:
# Clone the repo
!git clone https://<DRIVE>@github.com/java-master007/Adversarial-Representation-Learning-for-Medical-Imaging.git

In [ ]:
# Change to the correct directory
%cd Adversarial-Representation-Learning-for-Medical-Imaging/

In [ ]:
#! git checkout feature/pytorch_lightning_improvement

In [ ]:
# Install requirements
# It will need restart on colab
! pip install -r requirements.txt

## Generator

In [ ]:
%cd MedSinGAN

In [ ]:
# Import os to run experiments
import os

# Parameters to vary in generation mode
gen_test_stages = [6,8,10,12]
gen_test_lr = [0.1, 0.5]
gen_test_niter = [500, 1000, 2000]

In [ ]:
# Test all parameters to see improvements
for elem in gen_test_stages:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --train_stages ' + str(elem) + ' --gpu 0\n')
  
for elem in gen_test_lr:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --lr_scale ' + str(elem) + ' --gpu 0\n')

for elem in gen_test_niter:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --niter ' + str(elem) + ' --gpu 0\n')

### Benign Samples

In [ ]:
# Run a best combination for benign image
! python main_train.py --train_mode generation --input_name ../images/benign.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

In [ ]:
# Run a best combination for benign image 2
! python main_train.py --train_mode generation --input_name ../images/benign2.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

### Malign Samples

In [ ]:
# Run a best combination for malign image
! python main_train.py --train_mode generation --input_name ../images/malignant.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

In [ ]:
# Run a best combination for benign image 2
! python main_train.py --train_mode generation --input_name ../images/malignant_2.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

In [ ]:
# Run a best combination for benign image 2, with PreLU
! python main_train.py --train_mode generation --input_name ../images/malignant_2.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9 --activation prelu

## Download contents

In [ ]:
# Import files to download zips
from google.colab import files

In [ ]:
# Zip the mlruns metrics to analyse
!zip -r /content/mlrun.zip /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns
files.download("/content/mlrun.zip")

In [ ]:
# Zip the best model analysed based on the mlruns
!zip -r /content/best_generation_model.zip /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels
files.download("/content/best_generation_model.zip")